In [1]:
import numpy as np
import scipy.sparse
import IncSVD.src.IncSVD.EvolvingMatrix as EM
N = 1000
k = 64

### Row updates

In [2]:
data_init = scipy.sparse.rand(N, N, density=0.005).tocsr()
data_append = scipy.sparse.rand(N, int(N*0.5), density=0.005).tocsc() * 10
data_final = scipy.sparse.hstack([data_init, data_append]).tocsr()

In [3]:
M = EM.EvolvingMatrix(data_init, k, sparse=True, method="RPI")

Initial Uk matrix of evolving matrix set to shape of        (1000, 64).
Initial sigmak array of evolving matrix set to shape of     (64,).
Initial VHk matrix of evolving matrix set to shape of       (64, 1000).


In [4]:
M.add_column(data_append, l=200, t=10)

<class 'scipy.sparse._csc.csc_matrix'>


In [5]:
U0, S0, V0 = scipy.sparse.linalg.svds(data_init, k)
A0 = U0 @ np.diag(S0) @ V0
A0 = np.hstack([A0, np.zeros_like(data_append.toarray())])
res3 = np.linalg.norm((np.zeros_like(A0)-data_final))

In [6]:
Uk_true, Sigmak_true, Vk_true = scipy.sparse.linalg.svds(data_final, k) 
A1 = Uk_true @ np.diag(Sigmak_true) @ Vk_true
A2 = M.Uk @ np.diag(M.Sigmak) @ M.Vk.T
res1 = np.linalg.norm((A1-data_final))
res2 = np.linalg.norm((A2-data_final))
print(res1, res2, res3)

234.87410247066396 235.12391268096187 294.1743719265202


### Update weight

In [7]:
data_init = scipy.sparse.rand(N, N, density=0.005).tocsr()
data_plus1 = scipy.sparse.rand(N, 15, density=0.005).tocsc() * 10
data_plus2 = scipy.sparse.rand(N, 15, density=0.005).tocsc() * 10
data_final = data_init + data_plus1 @ data_plus2.T

In [8]:
M = EM.EvolvingMatrix(data_init, k, sparse=True, method="RPI")

Initial Uk matrix of evolving matrix set to shape of        (1000, 64).
Initial sigmak array of evolving matrix set to shape of     (64,).
Initial VHk matrix of evolving matrix set to shape of       (64, 1000).


In [9]:
M.update_weight(data_plus1, data_plus2, l=15, t=100)

<class 'scipy.sparse._csc.csc_matrix'>
<class 'scipy.sparse._csc.csc_matrix'>


In [10]:
Uk_true, Sigmak_true, Vk_true = scipy.sparse.linalg.svds(data_final, k) 
A1 = Uk_true @ np.diag(Sigmak_true) @ Vk_true
A2 = M.Uk @ np.diag(M.Sigmak) @ M.Vk.T
res1 = np.linalg.norm((A1-data_final))
res2 = np.linalg.norm((A2-data_final))
print(res1, res2)

35.50834107083238 36.07074336988177
